In [ ]:
! pip install autocorrect

In [1]:
# import the necessary libraries
import nltk
import string
import re
import requests
import pandas as pd
import json
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_excel('twitter_df.xlsx')  

# find 20 frequent emoji

In [3]:
#print (df["emojis"].nunique())
n = 20
top_20_emoji = df['emojis'].value_counts()[:n].index.tolist()
mask = df['emojis'].isin(top_20_emoji)
new_data = df[mask]
new_data = new_data.reset_index(drop=True)

#print(top_20_emoji)
dict={45:0,196:1,253:2,112:3,264:4,140:5,127:6,64:7,306:8,265:9,99:10,
         286:11,305:12,176:13,150:14,325:15,160:16,52:17,88:18,225:19}

new_data = new_data.replace({"emojis": dict})
new_data.insert(loc = 0,column = 'index',value = new_data.index)

# Text Preprocessing  functions

In [29]:
# 1- Clear out HTML characters 
import html
def clear_HTMLcharacters(text):
    text = html.unescape(text)
    return text


# 2- Encoding UTF-8 
def encode_utf8(text):
    text = text.encode('ascii','ignore')
    encode_tweet=text.decode(encoding='UTF-8')
    return encode_tweet


# 3- Extract Hashtags
def extract_hashtags(text):
    hashtag_list = []
    for word in text.split():
        if word[0] == '#':
            hashtag_list.append(word[1:])
    return hashtag_list


# 4- Removing URLs, Hashtags and Styles
def remove_URL_Hashtags_Style(text):  
    text = re.sub(r'https?:\/\/.\S+', "", text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    return text


# 5- Convert Numbers into words
import inflect
p = inflect.engine()
def convert_number(text):
    temp_str = text.split()
    new_string = []
    for word in temp_str:
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)
        else:
            new_string.append(word)
    temp_str = ' '.join(new_string)
    temp_str = re.sub(r'\d+', '', temp_str)
    return temp_str


# 6- Text Lowercase
def text_lowercase(text):
    return text.lower()


# 7- Replace Contraction 
Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will","'d":" would","'ve":" have","'re":" are"}
def replace_contraction(text):
    for key,value in Apos_dict.items():
            if key in text:
                text=text.replace(key,value)
    return text     


# 8- Split attached words
def split_attached_words(text):
    text = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",text) if s])
    return text


# 9- Remove More Than twice repeat letter
import itertools
def remove_more_than_twice(text):
    #One letter in a word should not be present more than twice in continuation
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    return text

# 10- Spell Checking
from autocorrect import Speller
spell = Speller(lang='en')
def spell_checking(text):
    text=spell(text)
    return text


# 11- Slang lookup
file=open("slang.txt","r")
slang=file.read()
slang=slang.split('\n')

def slang_lookup(text):
    text_tokens=text.split()
    slang_word=[]
    meaning=[]
    for line in slang:
        temp=line.split("=")
        slang_word.append(temp[0])
        meaning.append(temp[-1])
    for i,word in enumerate(text_tokens):
        if word in slang_word:
            idx=slang_word.index(word)
            text_tokens[i]=meaning[idx]
         
    text=" ".join(text_tokens)
    return text


# 12- Remove Punctuations
import string   
def remove_punctuations(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text
 
    

# 13- Remove whitespaces
def remove_whitespace(text):
    return  " ".join(text.split())
  
    
# 14- Remove  stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
en_stops = set(stopwords.words('english'))
def remove_stopwords(text):
    text_tokens = word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in en_stops]
    text = " ".join(filtered_text)
    text = re.sub(r'\d+', '', text)
    return text


# 15- Stemming
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()
def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    text  =" ".join(stems)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zebra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# total preprocess tweets

In [45]:
def preprocess_tweets(text):
    text = clear_HTMLcharacters(text)
    text = encode_utf8(text)
    hashtag_list = extract_hashtags(text)
    text = remove_URL_Hashtags_Style(text)
    text = split_attached_words(text)
    text = remove_more_than_twice(text)
    text = text_lowercase(text)
    text = convert_number(text)
    text = replace_contraction(text)
    text = remove_punctuations(text)      
    text = slang_lookup(text)      
    text = remove_whitespace(text)      
    #text = remove_stopwords(text)
    text = spell_checking(text)
    text = stem_words(text)
    return text ,hashtag_list


In [46]:
count = 0
for i in new_data["tweets"]:
    objects = []
    tweet = i
    #print(tweet)
    try:
        preprocess_tweet , hashtag_list = preprocess_tweets(tweet)
        #print(preprocess_tweet)
        new_data.loc[[count], "preprocess_tweets"] = preprocess_tweet
        print("tweet number ", count, "is preprocessed")
        new_data.loc[[count], "hashtags"] = (' '.join(hashtag_list))
        print("hashtags of tweet number ", count, "are fetched")
        count = count + 1
    except Exception as e:
        print(e)
        count = count + 1
        pass

tweet number  0 is preprocessed
hashtags of tweet number  0 are fetched
tweet number  1 is preprocessed
hashtags of tweet number  1 are fetched
tweet number  2 is preprocessed
hashtags of tweet number  2 are fetched
tweet number  3 is preprocessed
hashtags of tweet number  3 are fetched
tweet number  4 is preprocessed
hashtags of tweet number  4 are fetched
tweet number  5 is preprocessed
hashtags of tweet number  5 are fetched
tweet number  6 is preprocessed
hashtags of tweet number  6 are fetched
tweet number  7 is preprocessed
hashtags of tweet number  7 are fetched
tweet number  8 is preprocessed
hashtags of tweet number  8 are fetched
tweet number  9 is preprocessed
hashtags of tweet number  9 are fetched
tweet number  10 is preprocessed
hashtags of tweet number  10 are fetched
tweet number  11 is preprocessed
hashtags of tweet number  11 are fetched
tweet number  12 is preprocessed
hashtags of tweet number  12 are fetched
tweet number  13 is preprocessed
hashtags of tweet number 

tweet number  111 is preprocessed
hashtags of tweet number  111 are fetched
tweet number  112 is preprocessed
hashtags of tweet number  112 are fetched
tweet number  113 is preprocessed
hashtags of tweet number  113 are fetched
tweet number  114 is preprocessed
hashtags of tweet number  114 are fetched
tweet number  115 is preprocessed
hashtags of tweet number  115 are fetched
tweet number  116 is preprocessed
hashtags of tweet number  116 are fetched
tweet number  117 is preprocessed
hashtags of tweet number  117 are fetched
tweet number  118 is preprocessed
hashtags of tweet number  118 are fetched
tweet number  119 is preprocessed
hashtags of tweet number  119 are fetched
tweet number  120 is preprocessed
hashtags of tweet number  120 are fetched
tweet number  121 is preprocessed
hashtags of tweet number  121 are fetched
tweet number  122 is preprocessed
hashtags of tweet number  122 are fetched
tweet number  123 is preprocessed
hashtags of tweet number  123 are fetched
tweet number

tweet number  220 is preprocessed
hashtags of tweet number  220 are fetched
tweet number  221 is preprocessed
hashtags of tweet number  221 are fetched
tweet number  222 is preprocessed
hashtags of tweet number  222 are fetched
tweet number  223 is preprocessed
hashtags of tweet number  223 are fetched
tweet number  224 is preprocessed
hashtags of tweet number  224 are fetched
tweet number  225 is preprocessed
hashtags of tweet number  225 are fetched
tweet number  226 is preprocessed
hashtags of tweet number  226 are fetched
tweet number  227 is preprocessed
hashtags of tweet number  227 are fetched
tweet number  228 is preprocessed
hashtags of tweet number  228 are fetched
tweet number  229 is preprocessed
hashtags of tweet number  229 are fetched
tweet number  230 is preprocessed
hashtags of tweet number  230 are fetched
tweet number  231 is preprocessed
hashtags of tweet number  231 are fetched
tweet number  232 is preprocessed
hashtags of tweet number  232 are fetched
tweet number

tweet number  329 is preprocessed
hashtags of tweet number  329 are fetched
tweet number  330 is preprocessed
hashtags of tweet number  330 are fetched
tweet number  331 is preprocessed
hashtags of tweet number  331 are fetched
tweet number  332 is preprocessed
hashtags of tweet number  332 are fetched
tweet number  333 is preprocessed
hashtags of tweet number  333 are fetched
tweet number  334 is preprocessed
hashtags of tweet number  334 are fetched
tweet number  335 is preprocessed
hashtags of tweet number  335 are fetched
tweet number  336 is preprocessed
hashtags of tweet number  336 are fetched
tweet number  337 is preprocessed
hashtags of tweet number  337 are fetched
tweet number  338 is preprocessed
hashtags of tweet number  338 are fetched
tweet number  339 is preprocessed
hashtags of tweet number  339 are fetched
tweet number  340 is preprocessed
hashtags of tweet number  340 are fetched
tweet number  341 is preprocessed
hashtags of tweet number  341 are fetched
tweet number

tweet number  437 is preprocessed
hashtags of tweet number  437 are fetched
tweet number  438 is preprocessed
hashtags of tweet number  438 are fetched


In [47]:
new_data

,index,tweets,photos,emojis,preprocess_tweets,hashtags
0,0,"LIBRARIAN’S PICK: Faculty of Education and Social Sciences (FESS) #2 Title: Child Development Call No.: HQ767.9 .F45 2016 Edition: 7th -- Maklumat lanjut, hubungi:- 📞 03-32805081 / 010-8344100 (Kampus Betari Jaya) #unisellibrary #librarianspick https://t.co/WdboyDqatg",https://pbs.twimg.com/media/E-qpjZaUUAI3u5E.jpg,13,librarian pick faculti of educ and social scienc fess two titl child develop call no hq f two thousand and sixteen edit th maklumat layout hubungi campu beta jay unisellibrari librarianspick,2 unisellibrary librarianspick
1,1,Happy Labor Day 👮‍♀️👮👮‍♂️👷‍♀️👷👷‍♂️💂‍♀️💂💂‍♂️🕵️‍♀️🕵️🕵️‍♂️👩‍⚕️🧑‍⚕️👨‍⚕️👩‍🌾🧑‍🌾👨‍🌾👩‍🍳🧑‍🍳👨‍🍳👩‍🎓🧑‍🎓👨‍🎓👩‍🏫🧑‍🏫👨‍🏫👩‍💻🧑‍💻👨‍💻👩‍🔬🧑‍🔬👨‍🔬👩‍🎨🧑‍🎨👨‍🎨👩‍🚒🧑‍🚒👨‍🚒👩‍✈️🧑‍✈️👨‍✈️💃🕺❤️💙🤍🔴🔵⚪️🟥🟦⬜️🇺🇸 #LaborDayWeekend #LaborDay https://t.co/YjDJbQhQZ6,https://pbs.twimg.com/media/E-opbAXXsAgpNfz.jpg,0,happi labor day labor day weekend labor day,LaborDayWeekend LaborDay
2,2,@ApnaNewsAayega humko followers mangta🌚 Continue the chain!! Hum treat v denge🌚🤑 #AurrishtAsAnidita https://t.co/TP8ouKZXGd,https://pbs.twimg.com/media/E-qpl6ZVQAMgAwy.jpg,4,ana news aayega human follow manga continu the chain hum treat v dens aurrisht as acid,AurrishtAsAnidita
3,3,🇲🇾MY GO AB6IX 2ND ALBUM [MO’ COMPLETE] 📅Due: 23/9 1159PM 🎁Ktown4u Benefit: Random 1pc per album 💰RM78 ❌No 2nd payment ✅Add On RM5 for poster tube ✅Choose version 📎DM to order https://t.co/nypHPsOe4x,https://pbs.twimg.com/media/E-qqCYNUUAEWDUD.jpg,3,my go abi nd album mo complet due one thousand one hundr and fiftynin pm known benefit random pc per album rm no nd payment add on rm for poster tube choos version dm to order,
4,4,Our ERP Software enables you to maintain a perfect level of co-ordination among your different company operational processes. Telephone us today to get your ERP software. 📞: 0721722333/0733174743 📧: info@movetech.co.ke #movetechsolutions Raphael Tuju #MbogaZaUhuru https://t.co/0pOBoYRj17,https://pbs.twimg.com/media/E-qlVahXMAQ8XVM.jpg,13,our erp softwar enabl you to maintain a perfect level of coordin among your differ compani oper process telephon us today to get your erp softwar infomovetechcok movetechsolut raphael turn manga za hurt,movetechsolutions MbogaZaUhuru
...,...,...,...,...,...,...
434,434,"👨‍✈️ Captain's table - Fri 24th Sept @ 7:30pm A look back at the 2021 season incl. players of the year &amp; 🦆 trophy Open to anyone involved in senior cricket - incl juniors, umpires 🍟 £10 for entry - incl sausage &amp; chips + drink 📧 Chair of Cricket Smyth Snr to reserve #COYD🎯 https://t.co/ZGzy6p5Pdl",https://pbs.twimg.com/media/E-nkZunXIAIjiq4.jpg,0,captain is tabl fri th sept pm a look back at the two thousand and twentyon season incl player of the year trophi open to anyon involv in senior cricket incl junior umpir ten for entri incl sausag chip drink chair of cricket smith sr to reserv cold,COYD
435,435,September 6th is #LaborDay! Thank you to all those who labor. We appreciate each and every one of you! 🇺🇸 👮‍♀️👮‍♂️👷‍♀️👷‍♂️🕵️‍♀️🕵️‍♂️👩‍⚕️👨‍⚕️👩‍🌾👨‍🌾👩‍🍳👨‍🍳👩‍🏫👨‍🏫👩‍🏭👨‍🏭👩‍💻👨‍💻👩‍💼👨‍💼👩‍🔧👨‍🔧👩‍🔬👨‍🔬👩‍🚒👨‍🚒👩‍✈️👨‍✈️👩‍⚖ https://t.co/E3UhFp7rAX,https://pbs.twimg.com/media/E-m3JVNXEAQYCtM.jpg,0,septemb th is labor day thank you to all those who labor we appreci each and everi one of you,LaborDay!
436,436,@VonnyR7 Worked 😍 https://t.co/qjI6a6muAY,https://pbs.twimg.com/media/E-jC0tgXsAAmDl0.jpg,5,jonni r work,
437,437,The Xmas 🎅 Gift came way too early ! This one is heavy ! Anthem for Xmas &amp; beyond ! Edey Knack 🤯 ! Drops this week ! Thanks for the patience! ❤️🎵 #whobarberthebarber https://t.co/ybyQ1dflkz,https://pbs.twimg.com/media/E-qeioLWUAQ9Q9H.jpg,1,the ma gift came way too earli thi one is heavi anthem for ma beyond eden knack drop thi week thank for the patienc whobarberthebarb,whobarberthebarber


# Find Image classes with EfficientNet 

In [48]:
import torch
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7')
model.eval()

# image preprocessing
tfms = transforms.Compose([transforms.Resize(224),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

# Load ImageNet class names
labels_map = json.load(open('labels.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

Loaded pretrained weights for efficientnet-b7


In [50]:
from PIL import Image
from io import BytesIO

count = 0
for i in new_data["photos"]:
    objects = []
    url = i
    print(count)
    print(url)
    try:
        #esponse = requests.get(url)
        #img = Image.open(BytesIO(response.content))
        img = Image.open(f'./img/new_image_{count}.png')
        img = tfms(img).unsqueeze(0)
        with torch.no_grad():
             outputs = model(img)
        for idx in torch.topk(outputs, k=10).indices.squeeze(0).tolist():
            prob = torch.softmax(outputs, dim=1)[0, idx].item()
            label=labels_map[idx]
            objects.append(label)
        print(objects)
        new_data.loc[[count], "object"] = (' '.join(objects))
        count = count + 1
    except Exception as e:
        print(e)
        count = count + 1
        pass
        


0
https://pbs.twimg.com/media/E-qpjZaUUAI3u5E.jpg
['web site, website, internet site, site', 'printer', 'space shuttle', 'photocopier', 'airship, dirigible', 'sewing machine', 'missile', 'book jacket, dust cover, dust jacket, dust wrapper', 'screen, CRT screen', 'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM']
1
https://pbs.twimg.com/media/E-opbAXXsAgpNfz.jpg
['book jacket, dust cover, dust jacket, dust wrapper', 'comic book', 'theater curtain, theatre curtain', 'stage', 'torch', 'web site, website, internet site, site', 'bearskin, busby, shako', 'barbell', 'ballplayer, baseball player', 'basketball']
2
https://pbs.twimg.com/media/E-qpl6ZVQAMgAwy.jpg
['plate', 'tray', 'potpie', 'confectionery, confectionary, candy store', 'Petri dish', 'grocery store, grocery, food market, market', 'hot pot, hotpot', 'wok', 'Dutch oven', 'mixing bowl']
3
https://pbs.twimg.com/media/E-qqCYNUUAEWDUD.jpg
['switch, electric switch,

['shoe shop, shoe-shop, shoe store', 'tobacco shop, tobacconist shop, tobacconist', 'toyshop', 'library', 'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM', 'bookshop, bookstore, bookstall', 'turnstile', 'trench coat', 'grocery store, grocery, food market, market', 'wig']
28
https://pbs.twimg.com/media/E-qruEdUUAEUSNL.jpg
['web site, website, internet site, site', 'hand-held computer, hand-held microcomputer', 'monitor', 'desktop computer', 'tape player', 'cassette player', 'analog clock', 'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM', 'digital clock', 'potpie']
29
https://pbs.twimg.com/media/E-qnuo0XIAAzo7o.jpg
['web site, website, internet site, site', 'hand-held computer, hand-held microcomputer', 'oscilloscope, scope, cathode-ray oscilloscope, CRO', 'desktop computer', 'cash machine, cash dispenser, automated teller machine, automat

['basketball', 'drum, membranophone, tympan', 'whistle', "academic gown, academic robe, judge's robe", 'panpipe, pandean pipe, syrinx', 'microphone, mike', 'torch', 'harmonica, mouth organ, harp, mouth harp', 'vestment', 'stage']
55
https://pbs.twimg.com/media/E-qjERAUYAAC9d_.jpg
['web site, website, internet site, site', 'menu', 'analog clock', 'envelope', 'rule, ruler', 'crossword puzzle, crossword', 'Sussex spaniel', 'jean, blue jean, denim', 'toilet tissue, toilet paper, bathroom tissue', 'jersey, T-shirt, tee shirt']
56
https://pbs.twimg.com/media/E-qlRVmWYAEdvlh.jpg
['wig', 'hoopskirt, crinoline', 'pedestal, plinth, footstall', 'overskirt', 'shower cap', 'gown', 'lampshade, lamp shade', 'cuirass', 'maillot, tank suit', 'oboe, hautboy, hautbois']
57
https://pbs.twimg.com/media/E-qrBMxVEAIjCAX.jpg
['envelope', 'packet', 'book jacket, dust cover, dust jacket, dust wrapper', 'bookshop, bookstore, bookstall', 'carton', 'binder, ring-binder', 'library', 'rubber eraser, rubber, pencil e

['web site, website, internet site, site', 'hand-held computer, hand-held microcomputer', 'menu', 'oscilloscope, scope, cathode-ray oscilloscope, CRO', 'analog clock', 'monitor', 'screen, CRT screen', 'crossword puzzle, crossword', 'desktop computer', 'seashore, coast, seacoast, sea-coast']
83
https://pbs.twimg.com/media/E-lnkJmXMAUIvrD.jpg
['shoe shop, shoe-shop, shoe store', 'confectionery, confectionary, candy store', 'tobacco shop, tobacconist shop, tobacconist', 'toyshop', 'grocery store, grocery, food market, market', 'shopping cart', 'pinwheel', 'carton', 'clog, geta, patten, sabot', 'shopping basket']
84
https://pbs.twimg.com/media/E-p5TZjUYAED-HR.jpg
['lakeside, lakeshore', 'maze, labyrinth', 'library', 'patio, terrace', 'valley, vale', 'seashore, coast, seacoast, sea-coast', 'bell cote, bell cot', 'worm fence, snake fence, snake-rail fence, Virginia fence', 'pot, flowerpot', 'moving van']
85
https://pbs.twimg.com/media/E-opbAXXsAgpNfz.jpg
['book jacket, dust cover, dust jacke

['web site, website, internet site, site', 'bathing cap, swimming cap', 'envelope', 'pick, plectrum, plectron', 'balloon', 'golf ball', 'croquet ball', 'ping-pong ball', 'screen, CRT screen', 'punching bag, punch bag, punching ball, punchball']
111
https://pbs.twimg.com/media/E-qqKPKVkAMIHl1.jpg
['ballplayer, baseball player', 'knee pad', 'maillot', 'military uniform', 'ski', 'sweatshirt', 'bobsled, bobsleigh, bob', 'scoreboard', 'web site, website, internet site, site', 'jersey, T-shirt, tee shirt']
112
https://pbs.twimg.com/media/E-qscWgVQAEhI4o.jpg
['menu', 'web site, website, internet site, site', 'envelope', 'sunscreen, sunblock, sun blocker', 'screen, CRT screen', 'packet', 'diaper, nappy, napkin', 'safety pin', 'lotion', 'binder, ring-binder']
113
https://pbs.twimg.com/tweet_video_thumb/E-qpDGlVgAI9amP.jpg
['maillot', 'maillot, tank suit', 'bathing cap, swimming cap', 'gown', 'overskirt', 'sarong', 'bikini, two-piece', 'hoopskirt, crinoline', 'marimba, xylophone', 'mountain bike

...

['web site, website, internet site, site', 'menu', 'radio, wireless', 'screen, CRT screen', 'digital clock', 'hand-held computer, hand-held microcomputer', 'analog clock', 'oscilloscope, scope, cathode-ray oscilloscope, CRO', 'monitor', 'desktop computer']
310
https://pbs.twimg.com/media/E-qsM5nWEAU3VTW.jpg
['microphone, mike', 'cellular telephone, cellular phone, cellphone, cell, mobile phone', 'digital watch', 'stage', 'balance beam, beam', 'radio, wireless', 'digital clock', 'remote control, remote', 'television, television system', 'dumbbell']
311
https://pbs.twimg.com/media/E-qp9ucUYAIWKRd.jpg
['web site, website, internet site, site', 'menu', 'crossword puzzle, crossword', 'screen, CRT screen', 'monitor', 'analog clock', 'desktop computer', 'oscilloscope, scope, cathode-ray oscilloscope, CRO', 'television, television system', 'binder, ring-binder']
312
https://pbs.twimg.com/media/E-qZsFTWUAAR2GG.jpg
['lakeside, lakeshore', 'valley, vale', 'boathouse', 'canoe', 'dam, dike, dyke', 

['comic book', 'Band Aid', 'packet', 'hand-held computer, hand-held microcomputer', 'remote control, remote', 'web site, website, internet site, site', 'hard disc, hard disk, fixed disk', 'stopwatch, stop watch', 'slot, one-armed bandit', 'beer bottle']
338
https://pbs.twimg.com/media/E-qX31AWEAIZh8j.jpg
['sock', 'shoe shop, shoe-shop, shoe store', 'jean, blue jean, denim', 'clog, geta, patten, sabot', 'cowboy boot', 'knee pad', 'Loafer', 'running shoe', 'puck, hockey puck', 'ski']
339
https://pbs.twimg.com/media/E-qrYLgX0AI13wJ.jpg
['web site, website, internet site, site', 'space shuttle', 'comic book', 'volcano', 'book jacket, dust cover, dust jacket, dust wrapper', 'fire screen, fireguard', 'geyser', 'missile', 'stage', 'projectile, missile']
340
https://pbs.twimg.com/media/E-qnvyQXMAEuA91.png
['web site, website, internet site, site', 'bathing cap, swimming cap', 'envelope', 'pick, plectrum, plectron', 'balloon', 'golf ball', 'croquet ball', 'ping-pong ball', 'screen, CRT screen',

['seashore, coast, seacoast, sea-coast', 'beacon, lighthouse, beacon light, pharos', 'traffic light, traffic signal, stoplight', 'breakwater, groin, groyne, mole, bulwark, seawall, jetty', 'pier', 'lakeside, lakeshore', 'crane', 'water tower', 'parking meter', 'dam, dike, dyke']
368
https://pbs.twimg.com/media/E-qp-D_XoAE0wJz.jpg
['web site, website, internet site, site', 'digital clock', 'iPod', 'television, television system', 'analog clock', 'hand-held computer, hand-held microcomputer', 'remote control, remote', 'radio, wireless', 'cellular telephone, cellular phone, cellphone, cell, mobile phone', 'stopwatch, stop watch']
369
https://pbs.twimg.com/media/E-oTRcfX0AE_ycT.jpg
['tobacco shop, tobacconist shop, tobacconist', 'bakery, bakeshop, bakehouse', 'shoe shop, shoe-shop, shoe store', 'bathing cap, swimming cap', 'bookshop, bookstore, bookstall', 'military uniform', 'toyshop', 'grocery store, grocery, food market, market', 'assault rifle, assault gun', 'book jacket, dust cover, d

['web site, website, internet site, site', 'envelope', 'digital clock', 'cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM', 'screen, CRT screen', 'radio, wireless', 'analog clock', 'notebook, notebook computer', 'television, television system', 'oscilloscope, scope, cathode-ray oscilloscope, CRO']
396
https://pbs.twimg.com/media/E-qsYTkVgAAh83S.jpg
['web site, website, internet site, site', 'airship, dirigible', 'theater curtain, theatre curtain', 'stage', 'cinema, movie theater, movie theatre, movie house, picture palace', 'maze, labyrinth', 'planetarium', 'balloon', 'spotlight, spot', 'digital clock']
397
https://pbs.twimg.com/media/E99ZXoTWUAc2KoF.jpg
['binder, ring-binder', 'analog clock', 'book jacket, dust cover, dust jacket, dust wrapper', 'packet', 'comic book', 'magnetic compass', 'web site, website, internet site, site', 'wall clock', 'envelope', 'carton']
398
https://pbs.twimg.com/media/E-qrvITUcAEwuti.

['dumbbell', 'lens cap, lens cover', 'dial telephone, dial phone', 'hand blower, blow dryer, blow drier, hair dryer, hair drier', 'combination lock', 'barbell', 'microphone, mike', 'mouse, computer mouse', 'face powder', 'safe']
423
https://pbs.twimg.com/media/E-qU0sqUcAM_uf1.jpg
['patio, terrace', 'seashore, coast, seacoast, sea-coast', 'beacon, lighthouse, beacon light, pharos', 'monastery', 'tile roof', 'boathouse', 'bannister, banister, balustrade, balusters, handrail', 'sandbar, sand bar', 'crane', 'restaurant, eating house, eating place, eatery']
424
https://pbs.twimg.com/media/E-qg_rIVIAI20Yo.jpg
['web site, website, internet site, site', 'desktop computer', 'monitor', 'comic book', 'menu', 'notebook, notebook computer', 'screen, CRT screen', 'analog clock', 'television, television system', 'envelope']
425
https://pbs.twimg.com/media/E-qoJriVEAM_F2t.jpg
['solar dish, solar collector, solar furnace', 'maypole', 'banjo', 'basketball', 'radio telescope, radio reflector', 'unicycle,

In [53]:
new_data.to_excel("final_data.xlsx")